# Projet 10 : Evaluation production

Problématique : Créer un chatbot

+++++ expliquer Luis

Nous disposons des dialogues Frames au format JSON.

Les données proviennent de ce site : https://www.microsoft.com/en-us/research/project/frames-dataset/download/

In [45]:
import pandas as pd

In [46]:
import sys
# Insert the path of modules folder
sys.path.insert(0, "Package")

# Import the module0 directly since
import Scripts_Analyse01 as pk
import Scripts_AnalyseMulti01 as pk_multi
import Scripts_AnalyseMulti02_Unsupervised as pk_multi_unsp
import Scripts_AnalyseMulti02_Supervised as pk_multi_sup

## Envoies des données à Luis

In [60]:
from azure.cognitiveservices.language.luis.authoring import LUISAuthoringClient
from azure.cognitiveservices.language.luis.authoring.models import ApplicationCreateObject
from azure.cognitiveservices.language.luis.runtime import LUISRuntimeClient
from msrest.authentication import CognitiveServicesCredentials
from functools import reduce

import json, time, uuid

In [598]:
authoringKey = '33acfaaa00644a1cb49e5a30d16c32e5'
authoringEndpoint = 'https://luisappproject10weu0001-authoring.cognitiveservices.azure.com/'
predictionKey = '54489d2d37f94785b4cee624518f31b8'
predictionEndpoint = 'https://luis-appproject10-weu-0001.cognitiveservices.azure.com/'

In [599]:
client = LUISAuthoringClient(authoringEndpoint, CognitiveServicesCredentials(authoringKey))

In [602]:
app_id = "23e77110-61c4-4828-9a79-4edf3828e905"
app_name = "APP-PROJECT10CHATBOTFLY-WEU-001"
version_id = 0.1

In [ ]:
# define machine-learned entity
mlEntityDefinition = [
{
    "name": "dst_city",
    "children": [
        { "name": "city" }
    ]
},
{
    "name": "or_city",
    "children": [
        { "name": "city" }
    ]
},
{
    "name": "str_date",
    "children": [
        { "name": "date" }
    ]
},
{
    "name": "end_date",
    "children": [
        { "name": "date" }
    ]
},
{
    "name": "budget",
    "children": [
        { "name": "value" }
    ]
}]


## EVALUER LE MODELE

Réalisons les prédictions pour les données test

Pour cela, créons une fonction

In [623]:
runtimeCredentials = CognitiveServicesCredentials(predictionKey)
clientRuntime = LUISRuntimeClient(endpoint=predictionEndpoint, credentials=runtimeCredentials)

In [624]:
def luis_prediction(text):
    # Production == slot name
    predictionRequest = { "query" : text }
    predictionResponse = clientRuntime.prediction.get_slot_prediction(app_id, "Production", predictionRequest)
    return predictionResponse.prediction.entities

In [631]:
def split_pred(item, parentid):
    if item is None :
        print("None")
    else:
        if 'Book Flight' in item:
            itembk = item['Book Flight'][0]
            lab = mlEntityDefinition[parentid]
            parent = lab["name"]
            child = lab["children"][0]["name"]
            if parent in str(item):
                #print(itembk[parent])
                return itembk[parent][0][child][0]
            else:
                print("None")
        else:
            print("None")
                

Comparons les entités

In [462]:
mlEntityDefinition

[{'name': 'dst_city', 'children': [{'name': 'city'}]},
 {'name': 'or_city', 'children': [{'name': 'city'}]},
 {'name': 'str_date', 'children': [{'name': 'date'}]},
 {'name': 'end_date', 'children': [{'name': 'date'}]},
 {'name': 'budget', 'children': [{'name': 'value'}]}]

In [645]:
def compare_entity(item):
    score = []
    if item["pred_dst_city"]== item["dst_city"]:
        score.append(1)
    if item["pred_or_city"]== item["or_city"]:
        score.append(1)
    if item["pred_str_date"]== item["str_date"]:
        score.append(1)
    if item["pred_end_date"]== item["end_date"]:
        score.append(1)
    if item["pred_budget"]== item["budget"]:
        score.append(1)
    return np.sum(score)/5

    

Calculons un taux de reussite sur le nombre d'entités correcte

In [646]:
df_test["score"] = df_test.apply(compare_entity, axis=1)

Faisons une moyenne du taux de reussite

In [647]:
np.average(df_test["score"])

0.7746958637469585

## Prédiction 

In [648]:
runtimeCredentials = CognitiveServicesCredentials(predictionKey)
clientRuntime = LUISRuntimeClient(endpoint=predictionEndpoint, credentials=runtimeCredentials)

In [649]:
# Production == slot name
predictionRequest = { "query" : "Hi I'd like to go to Caprica from Paris, between Sunday August 21, 2016 and Wednesday August 31, 2016" }

predictionResponse = clientRuntime.prediction.get_slot_prediction(app_id, "Production", predictionRequest)
print("Top intent: {}".format(predictionResponse.prediction.top_intent))
print("Sentiment: {}".format (predictionResponse.prediction.sentiment))

for intent in predictionResponse.prediction.intents:
    print("\t{}".format (json.dumps (intent)))
print("Entities: {}".format (predictionResponse.prediction.entities))

Top intent: bookFlight
Sentiment: None
	"bookFlight"
Entities: {'Book Flight': [{'dst_city': [{'city': ['Caprica']}], 'or_city': [{'city': ['Paris,']}], 'str_date': [{'date': ['Sunday August 21, 2016']}]}], 'geographyV2': [{'value': 'Paris', 'type': 'city'}], 'datetimeV2': [{'type': 'daterange', 'values': [{'timex': '(2016-08-21,2016-08-31,P10D)', 'resolution': [{'start': '2016-08-21', 'end': '2016-08-31'}]}]}], 'number': [21, 2016, 31, 2016]}


## App insight

### Récupération des evènements du bot

In [13]:
import requests
import json

appId = "ef2a87f7-8613-4969-a421-818716f9ec1b"
appKey = "86cq5geh0yu3lxj4p5vxhhvphqnbbtk0721kow8q"

query = """
  customEvents
  | where timestamp > ago(1d)
  | order by timestamp
  | limit 10
"""

params = {"query": query}
headers = {'X-Api-Key': appKey}
url = f'https://api.applicationinsights.io/v1/apps/{appId}/query'

response = requests.get(url, headers=headers, params=params)

logs = json.loads(response.text)

for row in logs['tables'][0]['rows']:
  structured = dict(zip(logs['tables'][0], row))
  print(structured)

{'name': '2022-11-04T08:30:24.13063Z', 'columns': '1 - Bot recap confirmed by user', 'rows': 'customEvent'}
{'name': '2022-11-04T08:30:17.790642Z', 'columns': 'WaterfallStep', 'rows': 'customEvent'}
{'name': '2022-11-04T08:30:17.787646Z', 'columns': 'BotMessageReceived', 'rows': 'customEvent'}
{'name': '2022-11-04T08:30:15.5435Z', 'columns': 'BotMessageSend', 'rows': 'customEvent'}
{'name': '2022-11-04T08:30:15.539517Z', 'columns': 'BotMessageSend', 'rows': 'customEvent'}
{'name': '2022-11-04T08:30:15.539517Z', 'columns': 'WaterfallComplete', 'rows': 'customEvent'}
{'name': '2022-11-04T08:30:15.539517Z', 'columns': 'WaterfallStep', 'rows': 'customEvent'}
{'name': '2022-11-04T08:30:15.539517Z', 'columns': 'WaterfallStep', 'rows': 'customEvent'}
{'name': '2022-11-04T08:30:15.278471Z', 'columns': 'BotMessageReceived', 'rows': 'customEvent'}
{'name': '2022-11-04T08:30:12.989996Z', 'columns': 'BotMessageSend', 'rows': 'customEvent'}


In [12]:
for row in logs['tables'][0]['rows']:
  structured = dict(zip(logs['tables'][0], row))
  if "Bot recap" in structured["columns"]:
    print(structured["columns"])

{'name': '2022-11-04T08:20:46.300372Z', 'columns': '0 - Bot recap not confirmed by user', 'rows': 'customEvent'}
{'name': '2022-11-04T08:20:46.300372Z', 'columns': '0 - Bot recap not confirmed by user', 'rows': 'customEvent'}


### Répartition des confirmations/infirmations des réponses du bot par les utilisateurs (J-1)

In [21]:
import requests
import json

appId = "ef2a87f7-8613-4969-a421-818716f9ec1b"
appKey = "86cq5geh0yu3lxj4p5vxhhvphqnbbtk0721kow8q"

query = """
customEvents
| where name contains "Bot recap" and timestamp > ago(1d)
| summarize Count=sum(itemCount) by RequestName = name
"""

params = {"query": query}
headers = {'X-Api-Key': appKey}
url = f'https://api.applicationinsights.io/v1/apps/{appId}/query'

response = requests.get(url, headers=headers, params=params)

logs = json.loads(response.text)

for row in logs['tables'][0]['rows']:
  structured = dict(zip(logs['tables'][0], row))
  print(structured)

{'name': '1 - Bot recap confirmed by user', 'columns': 3}
{'name': '0 - Bot recap not confirmed by user', 'columns': 2}


### Nombre de demandes luis incomplètes par heure (J-1)

In [19]:
import requests
import json

appId = "ef2a87f7-8613-4969-a421-818716f9ec1b"
appKey = "86cq5geh0yu3lxj4p5vxhhvphqnbbtk0721kow8q"

query = """
customEvents
| where name contains "WaterFallComplete" and timestamp > ago(1d)
| summarize Count=sum(itemCount)  by bin(timestamp, 1h), RequestName = name
"""

params = {"query": query}
headers = {'X-Api-Key': appKey}
url = f'https://api.applicationinsights.io/v1/apps/{appId}/query'

response = requests.get(url, headers=headers, params=params)

logs = json.loads(response.text)

for row in logs['tables'][0]['rows']:
  structured = dict(zip(logs['tables'][0], row))
  print(structured)

{'name': '2022-11-04T03:00:00Z', 'columns': 'WaterfallComplete', 'rows': 3}
{'name': '2022-11-04T06:00:00Z', 'columns': 'WaterfallComplete', 'rows': 6}
{'name': '2022-11-04T08:00:00Z', 'columns': 'WaterfallComplete', 'rows': 12}


### Nombre de bugs du Bot (J-1)

In [22]:
import requests
import json

appId = "ef2a87f7-8613-4969-a421-818716f9ec1b"
appKey = "86cq5geh0yu3lxj4p5vxhhvphqnbbtk0721kow8q"

query = """
customEvents
 | where name contains "BotMessageSend" and customDimensions contains "text" and customDimensions["text"] contains "bug" and timestamp > ago(1d)
 | summarize count(itemCount) by bin(timestamp, 1h), RequestName = name
"""

params = {"query": query}
headers = {'X-Api-Key': appKey}
url = f'https://api.applicationinsights.io/v1/apps/{appId}/query'

response = requests.get(url, headers=headers, params=params)

logs = json.loads(response.text)

for row in logs['tables'][0]['rows']:
  structured = dict(zip(logs['tables'][0], row))
  print(structured)

{'name': '2022-11-04T03:00:00Z', 'columns': 'BotMessageSend', 'rows': 1}
